# 載入套件

In [41]:
# 爬蟲用
import requests
import re
import os
import urllib
from bs4 import BeautifulSoup
from urllib.request import urlretrieve
# 資料整理用
import pandas as pd


## 爬PTT標題

## 版的名稱 -> 網址
網址規則簡單，讓input從網址變成版的名稱

In [3]:
def get_board_url(Board_Name):
    url = urllib.parse.urljoin('https://www.ptt.cc/bbs/', Board_Name, '/index.html')
    return url

## 方便讓爬蟲自動往前爬

因為是從網頁板爬，要點上一頁(取得上一頁的連結)才會找到之前的文章。

In [4]:
def page_num(url, num):
    article = get_articles(url, today)
    bb = url
    for i in range(1, num+1):
        if i != 0:
            lastpage_url = get_lastpage(bb)
            df = get_articles(lastpage_url, today)
            article = article.append(df, axis)
    return article

In [5]:
def get_lastpage(url):
    
    r = requests.get(url, "html.parser", cookies={'over18': '1'})
    r.encoding = "UTF-8"
    soup = BeautifulSoup(r.text, 'lxml')
    divs = soup.find_all("a", "btn wide")
    lastpage_url = urllib.parse.urljoin("https://www.ptt.cc", divs[1]["href"])
    return lastpage_url

## 確定網頁結構格式是一般的貼文

有時版規、公告、至底文或是原po修改文章時誤觸可能造成文章格式與一般的發文不同

先排一些BUG掉

In [6]:
def checkformat(soup, class_tag, data, index, link):
    
    try:
        content = soup.select(class_tag)[index].text
    except Exception as e:
        print('格式跑掉爬不到QQ')
        # print 'checkformat:',str(e)
        content = "no " + data
    return content

## 取得內文

In [7]:
def get_content(url):
    r = requests.get(url, "html.parser", cookies={'over18': '1'})
    r.encoding = "UTF-8"
    soup = BeautifulSoup(r.text, 'lxml')
    # content  文章內文
    try:
        content = soup.find(id="main-content").text
        target_content = u'※ 發信站: 批踢踢實業坊(ptt.cc),'
        content = content.split(target_content)
        content = content[0].split(checkformat(soup, '.article-meta-value', 'date', 3, url))
        main_content = content[1].replace('\n', '  ')
        # print 'content:',main_content
    except Exception as e:
        main_content = 'main_content error'
        print('格式跑掉爬不到QQ')
    return main_content


## 取得網頁版的文章標題、推文數、日期、推文連結

In [8]:
def get_articles(url):
    
    r = requests.get(url, "html.parser", cookies={'over18': '1'})
    r.encoding = "UTF-8"
    soup = BeautifulSoup(r.text, 'lxml')
    divs = soup.find_all("div", "r-ent")
    articles = []

    for i in range(0, len(divs)):
                   
        push = 0                
        if divs[i].find("div", "nrec").string:                    
            try:                    
                push = int(divs[i].find("div", "nrec").string)
            except ValueError:
                pass
        if  divs[12].find("div", "nrec").string == "爆":
            try:
                push = 120
            except ValueError:
                pass
        
                
   
        if divs[i]("a"):
            date = divs[i].find("div", "date").string.strip()
            href = urllib.parse.urljoin('https://www.ptt.cc/',divs[i].find('a')['href'])
            title = divs[i].find('a').string
            content  = get_content(href)
                     
                        
            articles.append({
                'date': date,
                'push': push,
                'title': title,
                'link': href,
                'content' : content
             })
    
        
    return pd.DataFrame(articles)[['date', 'push', 'title', 'link', 'content']]

## 組成同一個函數

可以取得網頁版的標題，加上取得上一頁的網址，就可以一直自動往前爬了。

In [9]:
def Crawler_PTT_Content(Board_Name , page, date = None):
    
    url  = get_board_url(Board_Name)
    article = get_articles(url)
    for i in range(1, page+1):
        print('現在正在往前爬第', i, '頁')
        if i != 0:
            lastpage_url = get_lastpage(url)
            df = get_articles(lastpage_url)
            article = article.append(df, ignore_index=True)   
            url = lastpage_url
            
    if date !=None:
        article = article[(article.date == date)]
           
    
    return article

## DEMO

In [10]:
Crawler_PTT_Content('Gossiping', 5)

格式跑掉爬不到QQ
格式跑掉爬不到QQ
現在正在往前爬第 1 頁
現在正在往前爬第 2 頁
現在正在往前爬第 3 頁
現在正在往前爬第 4 頁
現在正在往前爬第 5 頁


,date,push,title,link,content
0,6/27,3,[問卦] 推圖為什麼越來越少了？,https://www.ptt.cc/bbs/Gossiping/M.1530032230....,以前很多版的推文區都有推圖 不知道是什麼時候開始 推圖越來越少 不知不覺地不見了...
1,6/27,0,Re: [問卦] 有沒有Herman Miller椅子的八卦,https://www.ptt.cc/bbs/Gossiping/M.1530032249....,安安你好 上週特價全功能25999腦波衰落也入手滑了 https://i.i...
2,6/27,4,[問卦] 一早起來就想做愛的人是那種人,https://www.ptt.cc/bbs/Gossiping/M.1530032319....,是這樣的啦 本肥朋友傳了個很ㄎㄧㄤ的對話給我看 好像4什麼 「一早起來...
3,6/27,0,[問卦] 有沒有藍色也有過去式的八卦Blued?,https://www.ptt.cc/bbs/Gossiping/M.1530032324....,最近你水管一直除了手遊廣告外 在推薦我 ”blued” 這個東西 我英文...
4,6/27,2,Re: [新聞] 去大陸僅62K vs.留台灣33K 新鮮人選擇陷,https://www.ptt.cc/bbs/Gossiping/M.1530032335....,※ 引述《good900183 (Robert)》之銘言： : 1.媒體來源: : ...
5,6/27,1,Re: [ＦＢ] 王威中 抓到了！侯友宜逃稅,https://www.ptt.cc/bbs/Gossiping/M.1530032358....,※ 引述《rafle (飆了啦)》之銘言： : 標題: Re: [ＦＢ] 王威中 抓到...
6,6/27,0,[問卦] 日系綜藝跟韓系綜藝那個是好看??,https://www.ptt.cc/bbs/Gossiping/M.1530032364....,這種觀感問題 就跟哈密瓜一樣 有的人喜歡 有的人不喜歡 但我身為一個從小...
7,6/27,1,[問卦] 為什麼煮菜影片下面很愛提智障問題,https://www.ptt.cc/bbs/Gossiping/M.1530032377....,譬如很愛問也最常看到的就是 「如果不能吃辣的該怎麼辦？」 幹 那你就不要加辣椒...
8,6/27,4,[問卦] 這炒飯能拿到幾分,https://www.ptt.cc/bbs/Gossiping/M.1530032381....,https://i.imgur.com/ex1PYFS.jpg 這肥宅我中午炒的炒飯 ...
9,6/27,0,Re: [問卦] 真的有人敢在八卦板放自己照片嗎,https://www.ptt.cc/bbs/Gossiping/M.1530032422....,※ 引述《yorkyoung (約克羊)》之銘言： : ※ 引述《missingkid...


## 爬PTT推文

現在有了標題與連結，我們針對每個連結爬下面的推文。

In [21]:
def Crawler_PTT_Push(url):
    r = requests.get(url, "html.parser", cookies={'over18': '1'})
    r.encoding = "UTF-8"
    soup = BeautifulSoup(r.text, 'lxml')
    divs = soup.find_all("div", id = "main-content")
    msg = divs[0].find_all("div", "push")
    articles = []
    
    for i in range(0, len(msg)):
        
        if msg[i]("span"):
            try:
                push_tag = msg[i].find("span", "f1 hl push-tag").string.replace(' ', '')
            except AttributeError:
                push_tag = msg[i].find("span", "hl push-tag").string.replace(' ', '')
                pass
        
        time = msg[i].find("span", "push-ipdatetime").string.strip()
        message = msg[i].find("span", "f3 push-content").string
        articles.append({
            'time': time,
            'push': push_tag,
            'message': message,
        })
    return pd.DataFrame(articles)[['push', 'message', 'time']]

## 爬推文DEMO

In [23]:
Crawler_PTT_Push('https://www.ptt.cc/bbs/Stock/M.1529844863.A.6E8.html')

,push,message,time
0,推,: 要噴了,06/24 20:57
1,推,: CHT真的超爽,06/24 21:06
2,推,: 噴個鬼啊XD,06/24 21:06
3,推,: 快進場,06/24 21:09
4,推,: 以前就這樣啦@@,06/24 21:09
5,推,: 我以為對員工越壞的越會噴xD,06/24 21:19
6,推,: 真是血汗,06/24 21:29
7,推,: 中華電不會噴,06/24 21:30
8,推,: 之前從100噴到113 現在回檔了,06/24 22:32
9,→,: 爛公司 已報名,06/24 22:55


In [46]:
p = Crawler_PTT_Content('Stock', 512)

格式跑掉爬不到QQ
格式跑掉爬不到QQ
現在正在往前爬第 1 頁
現在正在往前爬第 2 頁
現在正在往前爬第 3 頁
現在正在往前爬第 4 頁
現在正在往前爬第 5 頁
現在正在往前爬第 6 頁
現在正在往前爬第 7 頁
現在正在往前爬第 8 頁
現在正在往前爬第 9 頁
現在正在往前爬第 10 頁
現在正在往前爬第 11 頁
現在正在往前爬第 12 頁
格式跑掉爬不到QQ
格式跑掉爬不到QQ
現在正在往前爬第 13 頁
現在正在往前爬第 14 頁
現在正在往前爬第 15 頁
格式跑掉爬不到QQ
格式跑掉爬不到QQ
現在正在往前爬第 16 頁
現在正在往前爬第 17 頁
現在正在往前爬第 18 頁
格式跑掉爬不到QQ
格式跑掉爬不到QQ
現在正在往前爬第 19 頁
現在正在往前爬第 20 頁
現在正在往前爬第 21 頁
現在正在往前爬第 22 頁
現在正在往前爬第 23 頁
現在正在往前爬第 24 頁
現在正在往前爬第 25 頁
現在正在往前爬第 26 頁
現在正在往前爬第 27 頁
現在正在往前爬第 28 頁
現在正在往前爬第 29 頁
現在正在往前爬第 30 頁
現在正在往前爬第 31 頁
現在正在往前爬第 32 頁
現在正在往前爬第 33 頁
現在正在往前爬第 34 頁
格式跑掉爬不到QQ
格式跑掉爬不到QQ
現在正在往前爬第 35 頁
現在正在往前爬第 36 頁
現在正在往前爬第 37 頁
現在正在往前爬第 38 頁
現在正在往前爬第 39 頁
現在正在往前爬第 40 頁
現在正在往前爬第 41 頁
現在正在往前爬第 42 頁
現在正在往前爬第 43 頁
現在正在往前爬第 44 頁
現在正在往前爬第 45 頁
現在正在往前爬第 46 頁
現在正在往前爬第 47 頁
現在正在往前爬第 48 頁
現在正在往前爬第 49 頁
現在正在往前爬第 50 頁
現在正在往前爬第 51 頁
現在正在往前爬第 52 頁
現在正在往前爬第 53 頁
現在正在往前爬第 54 頁
現在正在往前爬第 55 頁
現在正在往前爬第 56 頁
現在正在往前爬第 57 頁
現在正在往前爬第 58 頁
格式跑掉爬不到QQ
格式跑掉爬不到QQ
現在正在往前爬第 59 頁
現在正在往前爬第 60 頁
現在正在往前爬第 61 頁
現在正在往前爬第 62 頁
現在正在往前爬第 63 頁
現在正在往前爬

現在正在往前爬第 490 頁
現在正在往前爬第 491 頁
現在正在往前爬第 492 頁
現在正在往前爬第 493 頁
現在正在往前爬第 494 頁
現在正在往前爬第 495 頁
現在正在往前爬第 496 頁
格式跑掉爬不到QQ
格式跑掉爬不到QQ
現在正在往前爬第 497 頁
現在正在往前爬第 498 頁
現在正在往前爬第 499 頁
現在正在往前爬第 500 頁
現在正在往前爬第 501 頁
現在正在往前爬第 502 頁
現在正在往前爬第 503 頁
現在正在往前爬第 504 頁
現在正在往前爬第 505 頁
現在正在往前爬第 506 頁
現在正在往前爬第 507 頁
現在正在往前爬第 508 頁
現在正在往前爬第 509 頁
現在正在往前爬第 510 頁
現在正在往前爬第 511 頁
現在正在往前爬第 512 頁


In [56]:
p.to_json('ptt_data.json')